# SQLAlchemy


If you want to manipulate Databases, it's good for you to know Python library: `sql-alchemy`. In this course, you'll learn: 

* Read & Write Data with SQL commands
* Read data from your datalake and load it to a proper database 
* Read data from your database and load it to datalake 
* Use Pandas features to read & write sql database


## Introduction to SQLAlchemy 🧙‍♂️🧙‍♂️

SQLAlchemy is the best way to handle relational databases with python. It is good for you to know the basic principles as you grow as a Data Scientist as you will be handling databases in your career.

### Structure of the API 

SQLAlchemy is organized in layers

![](https://docs.sqlalchemy.org/en/13/_images/sqla_arch_small.png)

You have two ways of handling SQL databases with SQLAlchemy: 

* ORM: It stands for Object Relation Mapper where you'll be able to communicate with the database and create very flexible models.

* SQLAlchemy Core: This layer is more schema-centric and will allow you to very simply access and query your tables. 

Let's check-out core functionnalities of both these layers.

### Nota Bene - SQLlite

For the rest of this course, we'll be using SQLlite which is a very light version of SQL database that is built-in. Therefore, we won't have to create any production database like PostgreSQL or MySQL. However, please note that SQLlite is NOT made for production and that you'll be using DB (like PostgreSQL or MySQL) in a production environment.

### ORM 

ORM uses python classes and instances to create and manipulate databases. Let's see how it works. Before diving into creating tables, let's create a db by connecting to it.

#### Create a connection

In [1]:
from sqlalchemy import create_engine
engine = create_engine("sqlite:///:memory:", echo=True)

In the above code, we created a "fake" database where we'll be using our computer's memory to mimic what a real database would be. 

NB, if you were to use a production database such as PostgreSQL, you would be using a connection just like this: 

`postgresql+psycopg2://{dbuser}:{dbpass}@{dbhost}/{dbname}`

#### Create a table 

Let's now create a table. To do so, we'll be using a python class and instanciate it. 

In [2]:
# Let's instance a declarative base to be able to use our python class
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Let's define our table 
from sqlalchemy import Column, Integer, String 
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

    def __repr__(self):
        return "<User(name='{}', fullname='{}', nickname='{}')>".format(self.name, self.fullname, self.nickname)

Here we represented our table `users` by class. As you can see, it contains 4 columns: 

* id -> of type Integer
* name -> of type String 
* fullname -> of type String 
* nickname -> of type String 

the `__repr__` method simply states how the output will be formated when we'll be calling our attributes. 

We now need to create our table within our database, we can do this by using the `create_all` method from `Base.metadata` module. 

In [3]:
Base.metadata.create_all(engine)

2021-01-04 17:52:42,990 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-01-04 17:52:42,991 INFO sqlalchemy.engine.base.Engine ()
2021-01-04 17:52:42,992 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-01-04 17:52:42,993 INFO sqlalchemy.engine.base.Engine ()
2021-01-04 17:52:42,994 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2021-01-04 17:52:42,994 INFO sqlalchemy.engine.base.Engine ()
2021-01-04 17:52:42,996 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2021-01-04 17:52:42,997 INFO sqlalchemy.engine.base.Engine ()
2021-01-04 17:52:42,999 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2021-01-04 17:52:43,000 INFO sqlalchemy.engine.base.Engine ()
2021-01-04 17:52:43,001 INFO sqlalchemy.engine.base.Engine COMMIT


#### Insert values 

Let's insert values in our database 

In [5]:
ed_user = User(id=1, name='ed', fullname='Ed Jones', nickname='edsnickname')

# Access Full row 
print(ed_user)

# Access ed_user name 
name = ed_user.name
print("name: {}".format(name))

# Access ed_user nickname
nickname = ed_user.nickname 
print("nickname: {}".format(nickname))


<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>
name: ed
nickname: edsnickname


We created data! As you can see we can access each column values simply by declaring a `.column_name`. 

#### Persist values in db 

Eventhough we created values, we haven't saved it in our database. We can do it by opening a `Session`

In [10]:
# Initialize a sessionmaker 
from sqlalchemy.orm import sessionmaker 
Session = sessionmaker(bind=engine)

Here we created a sessionmaker which will allow us to talk to our database. The `bind` argument takes an `engine` as parameter which corresponds to our database. 

In [11]:
# Instanciate Session 
session = Session()

# Add values to db 
session.add(ed_user)

# Commit the results 
session.commit()

Good job! We added our first value inside our db 👏👏 Note that it is very important to use the `.commit()` method to actually persist the values you inserted when you called the `.add()` method. 

#### Query values from a database

Now that we have some data inside our database, we can query it simply by using the `session` instance.

In [59]:
# Query our table users
user = session.query(User)

# Output all the results 
user.all()

2020-06-02 15:11:16,448 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-06-02 15:11:16,450 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2020-06-02 15:11:16,450 INFO sqlalchemy.engine.base.Engine ()


[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>]

You can now also use any SQL statement if you want to run more complex queries

In [60]:
from sqlalchemy import text

# Create a statement 
statement = text("SELECT * FROM users where name=:name")
statement

As you can see, the query looks almost as a real SQL query. The only thing is the `:name` parameter in `where name=:name`.

This will allow us to insert values afterwards the following way: 

In [61]:
session.query(User).from_statement(statement).\
                    params(name="ed").\
                    all()

2020-06-02 15:11:18,954 INFO sqlalchemy.engine.base.Engine SELECT * FROM users where name=?
2020-06-02 15:11:18,955 INFO sqlalchemy.engine.base.Engine ('ed',)


[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>]

As you can see, we used this `params` method that allow you to specify an operator from which you could apply your filter. 

### SQLAlchemy Core 

If you don't like classes and the declarative approach. You can use the SQLAlchemy Core layer of the library. Let's start by creating a new database. 

In [12]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

#### Create Tables 

You can simply create tables the following way.

In [13]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
meta = MetaData()
students = Table(
           'students', meta, 
           Column('id', Integer, primary_key = True), 
           Column('name', String), 
           Column('lastname', String),
        )

addresses = Table(
           'addresses', meta, 
           Column('id', Integer, primary_key = True), 
           Column('email_address', String), 
           Column("student_id", None, ForeignKey("students.id"))
        )

As you can see, I simply declared two tables with the `Table` class. Pay attention to `meta` as well which basically contain all the metadata (additional information) necessary to create an actual table. 

Let's now `create_all` tables to actually create them within our database. 

In [14]:
meta.create_all(engine)

2021-01-04 18:25:47,193 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-01-04 18:25:47,194 INFO sqlalchemy.engine.base.Engine ()
2021-01-04 18:25:47,195 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-01-04 18:25:47,196 INFO sqlalchemy.engine.base.Engine ()
2021-01-04 18:25:47,197 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("students")
2021-01-04 18:25:47,198 INFO sqlalchemy.engine.base.Engine ()
2021-01-04 18:25:47,200 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("students")
2021-01-04 18:25:47,200 INFO sqlalchemy.engine.base.Engine ()
2021-01-04 18:25:47,201 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("addresses")
2021-01-04 18:25:47,202 INFO sqlalchemy.engine.base.Engine ()
2021-01-04 18:25:47,203 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("addresses")
2021-01-04 18:25:47,203 INFO sqlalchemy.engine.base.Engine ()
2021-01-04 1

#### Insert values

If you need to insert values, you can do it the following way  

In [15]:
ins = students.insert().values(id="1", name="Jack", lastname="Johnson")
ins

This hasn't really inserted values just yet. You will need to first:

1. Create a connecion to your db 
2. Execute the query

In [16]:
# Connect to the db
conn = engine.connect()

# Execute the query 
result = conn.execute(ins)

# Always close the connection
conn.close()
result

2021-01-04 18:28:16,860 INFO sqlalchemy.engine.base.Engine INSERT INTO students (id, name, lastname) VALUES (?, ?, ?)
2021-01-04 18:28:16,862 INFO sqlalchemy.engine.base.Engine ('1', 'Jack', 'Johnson')
2021-01-04 18:28:16,863 INFO sqlalchemy.engine.base.Engine COMMIT


If you need to insert multiple values, you can do so simply by specifying a list of dictionnaries 

In [17]:
values = [
    {'student_id': 1, 'email_address' : 'jack@yahoo.com'},
    {'student_id': 1, 'email_address' : 'jack@msn.com'}
]

# Better way to handle connection
with engine.connect() as conn:
    conn.execute(addresses.insert(), values)

2021-01-04 18:29:54,567 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, student_id) VALUES (?, ?)
2021-01-04 18:29:54,568 INFO sqlalchemy.engine.base.Engine (('jack@yahoo.com', 1), ('jack@msn.com', 1))
2021-01-04 18:29:54,569 INFO sqlalchemy.engine.base.Engine COMMIT


#### Query values 

If you need to query your database, you can do it via `text` method and use actual sql queries. 

In [18]:
from sqlalchemy.sql import text

# Create a statement 
stmt = text("SELECT students.id, addresses.id, students.name, addresses.email_address FROM students "
            "JOIN addresses ON students.id=addresses.student_id "
            "WHERE students.id = 1")\
            .columns(
                students.c.id,
                addresses.c.id,
                addresses.c.student_id,
                students.c.name,
                addresses.c.email_address
            )

with engine.connect() as conn:
    result = conn.execute(stmt)

2021-01-04 18:31:41,924 INFO sqlalchemy.engine.base.Engine SELECT students.id, addresses.id, students.name, addresses.email_address FROM students JOIN addresses ON students.id=addresses.student_id WHERE students.id = 1
2021-01-04 18:31:41,926 INFO sqlalchemy.engine.base.Engine ()


/opt/conda/lib/python3.8/site-packages/sqlalchemy/engine/result.py:565: SAWarning: Number of columns in textual SQL (5) is smaller than number of columns requested (4)
  util.warn(


What is important to notice is: 

* Each line of the query is represented by a string. If you query needs to be on several lines, you need to add a space (" ") before going to the next line

* The `.columns()` works like a `.format()` method. This is where you will be specifying the names of the columns, you want to insert.

* Finally, for each columns we specified `table_name.c.column_name`. This is how would select a column from a given table. 

Now if you want to check the actual results. You can use the `.fetchall` method that will output the results as a list. 

In [19]:
result.fetchall()

[(1, 1, 'Jack', 'jack@yahoo.com'), (1, 2, 'Jack', 'jack@msn.com')]

## Simple things out with Pandas 😌😌

SQLAlchemy is the fundamental knowledge when dealing with SQL databases. However, there are simpler way to insert and query databases that would work most of the time. And, this is with Pandas. 

### Read SQL Databases with Pandas 

Most of the time, you were using pandas with csv files or excel spreadsheets but you can also use this library with sql databases. You would use the `read_sql` method. 

In [97]:
import pandas as pd 

# Create a statement 
stmt = text("SELECT students.id, addresses.id, students.name, addresses.email_address FROM students "
            "JOIN addresses ON students.id=addresses.student_id "
            "WHERE students.id = 1")\
            .columns(
                students.c.id,
                addresses.c.id,
                addresses.c.student_id,
                students.c.name,
                addresses.c.email_address
            )

df = pd.read_sql(
        stmt,
        engine
    )

df.head()

2020-06-02 15:42:43,361 INFO sqlalchemy.engine.base.Engine SELECT students.id, addresses.id, students.name, addresses.email_address FROM students JOIN addresses ON students.id=addresses.student_id WHERE students.id = 1
2020-06-02 15:42:43,362 INFO sqlalchemy.engine.base.Engine ()


,id,id,name,email_address
0,1,1,Jack,jack@yahoo.com
1,1,2,Jack,jack@msn.com


Now you have a very nice DataFrame that you can easily manipulate! 

Note that we still used SQLAlchemy as we create a statement `stmt` as well as an `engine` for pandas to be able to create it. 

### Update your database with Pandas 

The same way you can read sql, you can also write sql with pandas. 

In [98]:
# Create a new column
df["great_new_column"] = 0
df.head()

,id,id,name,email_address,great_new_column
0,1,1,Jack,jack@yahoo.com,0
1,1,2,Jack,jack@msn.com,0


In [99]:
# Push this new dataframe to our sql database
df.to_sql(
    "brand_new_table",
    engine
)

2020-06-02 15:46:33,460 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("brand_new_table")
2020-06-02 15:46:33,461 INFO sqlalchemy.engine.base.Engine ()
2020-06-02 15:46:33,463 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("brand_new_table")
2020-06-02 15:46:33,464 INFO sqlalchemy.engine.base.Engine ()
2020-06-02 15:46:33,466 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE brand_new_table (
	"index" BIGINT, 
	id BIGINT, 
	name TEXT, 
	email_address TEXT, 
	great_new_column BIGINT
)


2020-06-02 15:46:33,467 INFO sqlalchemy.engine.base.Engine ()
2020-06-02 15:46:33,468 INFO sqlalchemy.engine.base.Engine COMMIT
2020-06-02 15:46:33,469 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_brand_new_table_index ON brand_new_table ("index")
2020-06-02 15:46:33,470 INFO sqlalchemy.engine.base.Engine ()
2020-06-02 15:46:33,471 INFO sqlalchemy.engine.base.Engine COMMIT
2020-06-02 15:46:33,474 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-06-02 15:46:33,477 INFO s

In [101]:
# Let's query it 
stmt = text("SELECT * FROM brand_new_table")
result = conn.execute(stmt)
result.fetchall()

2020-06-02 15:47:40,786 INFO sqlalchemy.engine.base.Engine SELECT * FROM brand_new_table
2020-06-02 15:47:40,788 INFO sqlalchemy.engine.base.Engine ()


[(0, 1, 'Jack', 'jack@yahoo.com', 0), (1, 2, 'Jack', 'jack@msn.com', 0)]

## Resources 

* [Introduction to SQLAlchemy](https://www.youtube.com/watch?v=woKYyhLCcnU)
* [SQLAlchemy Overview](https://docs.sqlalchemy.org/en/13/intro.html)
* [Object Relational Tutorial](https://docs.sqlalchemy.org/en/13/orm/tutorial.html)
* [SQL Expression Language Tutorial](https://docs.sqlalchemy.org/en/13/core/tutorial.html)
* [Pandas Read SQL](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html#pandas.read_sql)
* [Pandas To SQL](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html)